In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
books=pd.read_csv("/kaggle/input/book-recommendation-dataset/Books.csv")
ratings=pd.read_csv("/kaggle/input/book-recommendation-dataset/Ratings.csv")
users=pd.read_csv("/kaggle/input/book-recommendation-dataset/Users.csv")

In [ ]:
print(books.shape,ratings.shape,users.shape)

In [ ]:
books=books.drop(['Image-URL-S','Image-URL-M','Image-URL-L'],axis=1)
books.head()

In [ ]:
ratings.head()

In [ ]:
users.head()

In [ ]:
#checking how many ratings given by each user
user_rating_count = ratings.groupby('User-ID').count().sort_values(by = 'ISBN', ascending = False)
user_rating_count

In [ ]:
user_rating_count = user_rating_count[user_rating_count['ISBN']>100]
user_rating_count.shape

In [ ]:
#Get counts of the ratings for each book
book_rating_count = ratings.groupby('ISBN').count().sort_values(by = 'Book-Rating', ascending = False)
book_rating_count = book_rating_count[book_rating_count['User-ID'] > 50]
print(book_rating_count.shape)
book_rating_count[:10]

In [ ]:
ratings = ratings.merge(books, on = 'ISBN')


In [ ]:
ratings

In [ ]:
ratings = ratings[ratings['User-ID'].isin(user_rating_count.index)]
ratings = ratings[ratings['ISBN'].isin(book_rating_count.index)]

In [ ]:
ratings

In [ ]:
piv_table = ratings.pivot_table(columns = "User-ID", index = "ISBN", values = "Book-Rating")

In [ ]:
piv_table.shape  #as this is sparce matrix we need to fill none values with zero

In [ ]:
piv_table = piv_table.fillna(0)

In [ ]:
piv_table.iloc[:100, :100]

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
SVD=TruncatedSVD(n_components=1000)
matrix=SVD.fit_transform(piv_table)

In [ ]:
var_explained = SVD.explained_variance_ratio_.sum()
var_explained

In [ ]:
matrix.shape

In [ ]:
corr=np.corrcoef(matrix)
corr.shape 

In [ ]:
piv_table[piv_table.index == '000649840X']

In [ ]:
def recommenditions_dataframe(recs):
    bookids=[i[0] for i in recs]
    df1=pd.DataFrame(columns=['bookid','Title','Author'])
    for i in bookids:
        tmp=ratings[(ratings['ISBN']==i)].iloc[0]
        tmp=[tmp['ISBN'],tmp['Book-Title'],tmp['Book-Author']]
        df1.loc[len(df1.index)]=tmp[:]
    return df1

In [ ]:
def recommend(bookid):
    for idx,value in enumerate(piv_table.index):
        if value==bookid:
            tmp=[[v,i] for i,v in enumerate(corr[idx])]
            tmp=sorted(tmp,reverse=True)[:10]
            ans=[]
            for i in tmp:
                ans.append([piv_table.index[i[1]],i[0]])   #bookid,correlation 
            return ans
    return [['Book Not Found']]    

In [ ]:
rec=recommend('002542730X')   #bookid,correlation_value
#print(rec)
rec=recommenditions_dataframe(rec)
rec

In [ ]:
rec=recommend('0446527017')   #bookid,correlation_value
#print(rec)
rec=recommenditions_dataframe(rec)
rec

In [ ]:
rec=recommend("0684835959")   #bookid,correlation_value
#print(rec)
rec=recommenditions_dataframe(rec)
rec